In [ ]:
# Experiment 1

# -----------------------------------------------------------------------------
# state_transfer_rabi.py
# Purpose: simulate 1 electron + 2 nuclei flip-flop and visualize Rabi oscillation
# -----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import qutip
from qutip import SolverOptions

# ------------------------
# Configuration switches
# ------------------------
np.random.seed(0)  # for reproducible random hyperfine if used
USE_ESR = False
USE_NMR = False
USE_RND_HF = False   # if True, random heterogeneous a_j; else homogeneous a_j
USE_HF_D = True # use diagonal hyperfine

# ------------------------
# Parameters (suggested / realistic)
# ------------------------
MHz = 1e6
# Put Zeeman terms to zero to be on-resonance (Beff ~ 0)
#omega_eZ = 2*np.pi*50*MHz
#omega_NZ = 2*np.pi*5*MHz
B0 = 0.1  # With 0.1T, we have omega_eZ=2π*50MHz, omega_NZ=2π*5MHz
gmuB = 3.1415926536e9   # rad/s/T  (set so B0*gmuB = 2π*50MHz)
gnumun = 3.1415926536e8 # rad/s/T  (set so B0*gnumun = 2π*5MHz)
omega_eZ = gmuB * B0
omega_NZ = gnumun * B0

# Base hyperfine amplitude (rad/s). Taylor uses A ~ 90 ueV total; here we pick a
# smaller toy value that gives Rabi in ~0.5-1 MHz range for n=2 for demonstration.
a_mean = 2 * np.pi * 0.5 * MHz   # = 2π * 0.5 MHz  (rad/s)

# ESR / NMR (not used in resonant flip-flop demonstration)
omega_1_MW = 2*np.pi*0.05*MHz
omega_MW   = 2*np.pi*5*MHz
omega_1_RF  = 2*np.pi*0.02*MHz
omega_RF    = 2*np.pi*1*MHz

n_nuclei = 2

# ------------------------
# Spin operators and helper tensor ops
# ------------------------
sx, sy, sz = 0.5*qutip.sigmax(), 0.5*qutip.sigmay(), 0.5*qutip.sigmaz()
sp, sm = 0.5*qutip.sigmap(), 0.5*qutip.sigmam()

def e(op):
    return qutip.tensor([op] + [qutip.qeye(2) for _ in range(n_nuclei)])

def n(op, k):
    ops = [qutip.qeye(2)]*(1+n_nuclei)
    ops[k+1] = op
    return qutip.tensor(ops)

# ------------------------
# Hyperfine couplings
# ------------------------
if USE_RND_HF:
    a_list = a_mean * np.random.uniform(0.5, 1.5, n_nuclei)
else:
    a_list = [a_mean]*n_nuclei

# Analytical Rabi frequency estimate (collective)
Omega_est = np.sqrt(np.sum(np.abs(a_list)**2))   # rad/s
print("a_list:", a_list)
print("Analytical collective Rabi rate Omega/(2π) [Hz]:", Omega_est/(2*np.pi))

# ------------------------
# Hamiltonian pieces
# ------------------------
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))

# hyperfine: diagonal + flip-flop
#H_hf = sum(a_list[k] * ( e(sp)*n(sm,k) + e(sm)*n(sp,k) + e(sz)*n(sz,k) )
#           for k in range(n_nuclei))
H_D = sum(a_list[k] * e(sz)*n(sz,k) for k in range(n_nuclei))
H_Omega = sum(a_list[k] * ( e(sp)*n(sm,k) + e(sm)*n(sp,k) ) for k in range(n_nuclei))
H_hf = H_Omega
if USE_HF_D:
    H_hf += H_D

# drives
H_esr = [
    [omega_1_MW * e(sx), "cos(wmw*t)"],
    [-omega_1_MW * e(sy), "sin(wmw*t)"]
]
H_nmr = [[omega_1_RF * n(sx, k), "cos(wrf*t)"] for k in range(n_nuclei)]

# assemble H
H = [H_eZ + H_NZ + H_hf]
if USE_ESR:
    H += H_esr
if USE_NMR:
    H += H_nmr

# ------------------------
# Initial state: electron DOWN, all nuclei UP  => |↓>_e ⊗ |0...0>_n
# (we use basis: |0> = |↑>, |1> = |↓> in qutip's basis(2,0) is 'up')
# ------------------------
up = qutip.basis(2,0)
down = qutip.basis(2,1)

# nuclei basis states: |0>_n = |↑>_n (basis index 0)
nuclear_ground = qutip.tensor(up, up)

# Choose visible, non-equal superposition amplitudes
alpha = 0 # 4/5
beta  = 1 # 3/5
# normalize
norm = np.sqrt(alpha**2 + beta**2)
alpha /= norm
beta  /= norm


# Build |ψ(0)> = (α|↑> + β|↓>)_e  ⊗ |00>_n
psi0 = qutip.tensor(alpha*up + beta*down, up, up)

# ------------------------
# Time grid chosen based on Omega_est
# pick tmax ~ several Rabi periods
t_swap = np.pi / Omega_est if Omega_est>0 else 1e-6
tmax = 4 * t_swap
tgrid = np.linspace(0, tmax, 2000)
args = {"wmw": omega_MW, "wrf": omega_RF}

# ------------------------
# Expectation ops: electron spin components and projectors if desired
# ------------------------
S_ops = [e(sx), e(sy), e(sz)]
P_up_e = e(up*up.dag())
P_up_n = [ n(up*up.dag(), k) for k in range(n_nuclei) ]

e_ops = [P_up_e] + P_up_n + S_ops

# ------------------------
# Solve and store states
# ------------------------
result = qutip.sesolve(H, psi0, tgrid, e_ops=e_ops, args=args,
                       options=SolverOptions(store_states=True))
states_t = result.states

# unpack expectations
expect = np.array(result.expect)   # shape (n_ops, len(tgrid))
P_e_up_t = expect[0]
P_n_up_t = expect[1:1+n_nuclei]
Sx_t, Sy_t, Sz_t = expect[1+n_nuclei:1+n_nuclei+3]

# ------------------------
# Build the normalized collective single-flip nuclear state |1>_n
# For general a_list:
# |1>_n = (sum |a_j|^2)^(-1/2) * sum_j a_j |... j flipped ...>
# For n_nuclei=2, the nuclear basis order for tensor(n1,n2) is
#   |00>,|01>,|10>,|11>  (with |0>=up,|1>=down)
# The states with exactly one nucleus flipped from |00> are |01> and |10>.
# -----------------------------------------------------------------------
# build nuclear basis kets
n00 = qutip.tensor(up, up)
n01 = qutip.tensor(up, down)
n10 = qutip.tensor(down, up)
n11 = qutip.tensor(down, down)

# create the collective |1>_n vector
# Note: we must match ordering: coefficient for nucleus j refers to flipping nucleus j (indexing k=0->n1, k=1->n2)
# For k=0 -> flip n1 (gives |10>), for k=1 -> flip n2 (gives |01>)
coeffs = []
states_single_flip = []
for k in range(n_nuclei):
    if k == 0:
        states_single_flip.append(n10)   # flip nucleus 1
    elif k == 1:
        states_single_flip.append(n01)   # flip nucleus 2
    else:
        raise NotImplementedError("n_nuclei>2 not implemented in this explicit routine")
    coeffs.append(a_list[k])

norm = np.sqrt(np.sum(np.abs(coeffs)**2))
n1 = sum(c*s for c,s in zip(coeffs, states_single_flip)) / norm   # Qobj nuclear ket

# Full two many-body kets
phi_initial = qutip.tensor(down, n00)           # |↓>_e ⊗ |00>_n
phi_target  = qutip.tensor(up,   n1)            # |↑>_e ⊗ |1>_n  (collective)
phi_up_00 = qutip.tensor(up, n00)

# compute populations of these collective states over time (overlaps)
P_initial = np.zeros(len(states_t))
P_target  = np.zeros(len(states_t))
P_up00 = np.zeros(len(states_t))
for ti, psi in enumerate(states_t):
    P_initial[ti] = np.abs(phi_initial.overlap(psi))**2
    P_target[ti]  = np.abs(phi_target.overlap(psi))**2
    P_up00[ti] = np.abs(phi_up_00.overlap(psi))**2

# Also compute full 8 populations (for plotting)
probs = np.zeros((len(states_t), 2**(1+n_nuclei)))
for i, psi in enumerate(states_t):
    vec = psi.full().flatten()
    probs[i,:] = np.abs(vec)**2

# ------------------------
# PLOTTING section
# ------------------------
# 1) Rabi oscillation: initial <-> target collective populations
plt.figure(figsize=(8,4))
plt.plot(tgrid*1e6, P_initial, label=r"$P(|\downarrow\rangle_e\otimes|0\rangle_n)$")
plt.plot(tgrid*1e6, P_target,  label=r"$P(|\uparrow\rangle_e\otimes|1\rangle_n)$")
plt.plot(tgrid*1e6, P_up00, label=r"$P(|\uparrow\rangle_e\otimes|00\rangle_n)$")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Collective flip-flop Rabi oscillation (analytical Ω/2π={:.3f} MHz)".format(Omega_est/(2*np.pi)/1e6))
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2) full 8 basis populations
plt.figure(figsize=(9,5))
for k in range(2**(1+n_nuclei)):
    plt.plot(tgrid*1e6, probs[:,k], label=f"|{k:03b}>")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Populations of all 8 basis states (electron,n1,n2)")
plt.legend(loc='upper right', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

# 3) electron spin expectations
plt.figure(figsize=(8,4))
plt.plot(tgrid*1e6, Sx_t, label=r"$\langle S_x\rangle$")
plt.plot(tgrid*1e6, Sy_t, label=r"$\langle S_y\rangle$")
plt.plot(tgrid*1e6, Sz_t, label=r"$\langle S_z\rangle$")
plt.xlabel("time [µs]")
plt.ylabel("Spin expectation")
plt.title("Electron spin expectations")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 4) 2x4 amplitude heatmaps (initial and at time of max transfer)
def plot_amplitudes_matrix(psi, title="Amplitude matrix (probabilities)"):
    vec = psi.full().flatten()
    mat = vec.reshape((2, 2**n_nuclei))   # electron rows, nuclear columns
    fig, ax = plt.subplots(figsize=(5,2.5))
    im = ax.imshow(np.abs(mat)**2, interpolation='nearest', aspect='auto')
    plt.colorbar(im, ax=ax, label='Population')
    ax.set_xticks(np.arange(2**n_nuclei))
    # nuclear labels for n_nuclei=2:
    ax.set_xticklabels(["00","01","10","11"])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["↑e","↓e"])
    ax.set_xlabel("nucleus state (n1 n2)")
    ax.set_ylabel("electron")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

plot_amplitudes_matrix(states_t[0], title="Initial amplitudes")
imax = np.argmax(P_target)
plot_amplitudes_matrix(states_t[imax], title=f"Amplitudes at max target population (t={tgrid[imax]*1e6:.3f} µs)")

# 5) print diagnostics
print("Analytical Omega/(2π) [Hz]:", Omega_est/(2*np.pi))
print("Swap time t_swap (analytical) [µs]:", (np.pi / Omega_est)*1e6)
print("Max simulated fidelity (target population):", P_target.max())
print("Time of max (µs):", tgrid[np.argmax(P_target)]*1e6)
# -----------------------------------------------------------------------------


In [ ]:
# -----------------------------------------------------------------------------
# state_transfer_rabi_with_Bres.py
# Purpose: simulate 1 electron + 2 nuclei flip-flop and visualize Rabi oscillation
# - adds dynamic calculation of B0_res (resonant B field) per Taylor formula
# - keeps existing configuration switches for HF randomness, ESR/NMR, HF diagonal
# -----------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import qutip
from qutip import SolverOptions

# ------------------------
# Reproducibility / switches
# ------------------------
np.random.seed(0)
USE_ESR = False
USE_NMR = False
USE_RND_HF = False    # if True, random heterogeneous a_j; else homogeneous a_j
USE_HF_D = True       # include diagonal hyperfine term
USE_B_RES = False      # compute and use resonant B0 (from Taylor formula) if True

# ------------------------
# Physical / simulation parameters
# ------------------------
MHz = 1e6
n_nuclei = 2

# Initial (fallback) B0 [T] if USE_B_RES is False
B0_fallback = 0.1   # Tesla

# We'll pick gmuB and gnumun such that gmuB * B0 gives desired electron and nuclear Zeeman
# For simplicity: pick gmuB and gnumun values that give ~2π*50MHz and 2π*5MHz @ B0=0.1T.
# These are effective g*μ_B and g_n μ_n products (in rad/s/T).
gmuB = 2*np.pi*50e6 / 0.1     # rad/s/T -> yields ω_eZ = 2π*50MHz at B0=0.1 T
gnumun = 2*np.pi*5e6 / 0.1    # rad/s/T -> yields ω_nZ = 2π*5MHz at B0=0.1 T

# Base hyperfine amplitude (per-nucleus coupling, in rad/s)
# small toy value which gives Ω ~ 0.5 MHz for n=2
a_mean = 2 * np.pi * 0.5 * MHz   # rad/s

# ESR / NMR (not used here)
omega_1_MW = 2*np.pi*0.05*MHz
omega_MW   = 2*np.pi*5*MHz
omega_1_RF  = 2*np.pi*0.02*MHz
omega_RF    = 2*np.pi*1*MHz

# ------------------------
# Spin operators and tensor helpers
# ------------------------
sx, sy, sz = 0.5*qutip.sigmax(), 0.5*qutip.sigmay(), 0.5*qutip.sigmaz()
sp, sm = 0.5*qutip.sigmap(), 0.5*qutip.sigmam()

def e(op):
    return qutip.tensor([op] + [qutip.qeye(2) for _ in range(n_nuclei)])

def n(op, k):
    ops = [qutip.qeye(2)]*(1+n_nuclei)
    ops[k+1] = op
    return qutip.tensor(ops)

# ------------------------
# Hyperfine couplings
# ------------------------
if USE_RND_HF:
    a_list = a_mean * np.random.uniform(0.5, 1.5, n_nuclei)
else:
    a_list = [a_mean]*n_nuclei

# Collective Rabi estimate (analytic, simple): Ω = sqrt(sum_j |a_j|^2)
Omega_est = np.sqrt(np.sum(np.abs(a_list)**2))   # rad/s
print("a_list:", a_list)
print("Analytical collective Rabi rate Omega/(2π) [Hz]:", Omega_est/(2*np.pi))

# ------------------------
# Resonant B0 calculation (Taylor, approximate)
#  Taylor gives (for perfectly polarized nuclei):
#    δ = (g*μ_B - g_n μ_n) B0 + I0 A + (I0 - 1) A / N
#  Setting δ = 0 -> B0_res = [I0 A + (I0 - 1) A / N] / (g*μ_B - g_n μ_n)
#
#  Here we approximate:
#   - I0 is nuclear spin (use I0 = 1/2 for spin-1/2 nuclei)
#   - A is taken as the mean per-site coupling ā (Taylor uses A ~ total constant; we use ā
#     to keep consistent units). This is an approximation for small N.
# ------------------------
I0 = 0.5  # nuclear spin quantum number (spin-1/2)
N = float(n_nuclei)
a_bar = np.mean(a_list)   # approximate per-site coupling (rad/s)

# Taylor's formula uses A in same units as a_j; we use a_bar as proxy.
# Compute numerator and B0_res (in Tesla)
numerator = I0 * a_bar + (I0 - 1.0) * (a_bar / N)
denom = (gmuB - gnumun)
if denom == 0:
    B0_res = B0_fallback
else:
    B0_res = numerator / denom

# Choose which B0 to use
if USE_B_RES:
    B0 = B0_res
    print("Using resonant B0 (computed):", B0_res, "T")
else:
    B0 = B0_fallback
    print("Using fallback B0:", B0_fallback, "T")

# Zeeman energies (rad/s)
omega_eZ = gmuB * B0
omega_NZ = gnumun * B0
print("omega_eZ/(2π): {:.3f} MHz, omega_NZ/(2π): {:.3f} MHz".format(omega_eZ/(2*np.pi)/1e6, omega_NZ/(2*np.pi)/1e6))

# ------------------------
# Hamiltonian pieces
# ------------------------
H_eZ = omega_eZ * e(sz)
H_NZ = omega_NZ * sum(n(sz, k) for k in range(n_nuclei))

# Hyperfine: diagonal + flip-flop
H_D = sum(a_list[k] * e(sz)*n(sz,k) for k in range(n_nuclei))
H_Omega = sum(a_list[k] * ( e(sp)*n(sm,k) + e(sm)*n(sp,k) ) for k in range(n_nuclei))
H_hf = H_Omega
if USE_HF_D:
    H_hf += H_D

# drives (unused for this simple resonant demo)
H_esr = [
    [omega_1_MW * e(sx), "cos(wmw*t)"],
    [-omega_1_MW * e(sy), "sin(wmw*t)"]
]
H_nmr = [[omega_1_RF * n(sx, k), "cos(wrf*t)"] for k in range(n_nuclei)]

H = [H_eZ + H_NZ + H_hf]
if USE_ESR:
    H += H_esr
if USE_NMR:
    H += H_nmr

# ------------------------
# Initial state: (α |↑> + β |↓>)_e ⊗ |00>_n
# ------------------------
up = qutip.basis(2,0)
down = qutip.basis(2,1)

# Example non-equal amplitudes for visibility:
alpha = 0.8
beta  = 0.6
# normalize (so |α|^2 + |β|^2 = 1)
norm = np.sqrt(np.abs(alpha)**2 + np.abs(beta)**2)
alpha /= norm
beta  /= norm

psi0 = qutip.tensor(alpha*up + beta*down, up, up)   # electron in superposition, nuclei all up
print("Initial state norm check:", (psi0.dag()*psi0).full().item().real)

# ------------------------
# Time grid chosen based on Omega_est
# ------------------------
t_swap = np.pi / Omega_est if Omega_est>0 else 1e-6
tmax = 4 * t_swap
tgrid = np.linspace(0, tmax, 2000)
args = {"wmw": omega_MW, "wrf": omega_RF}

# ------------------------
# Expectation ops
# ------------------------
S_ops = [e(sx), e(sy), e(sz)]
P_up_e = e(up*up.dag())
P_up_n = [ n(up*up.dag(), k) for k in range(n_nuclei) ]
e_ops = [P_up_e] + P_up_n + S_ops

# ------------------------
# Solve and store states
# ------------------------
result = qutip.sesolve(H, psi0, tgrid, e_ops=e_ops, args=args,
                       options=SolverOptions(store_states=True))
states_t = result.states

# unpack expectations
expect = np.array(result.expect)   # shape (n_ops, len(tgrid))
P_e_up_t = expect[0]
P_n_up_t = expect[1:1+n_nuclei]
Sx_t, Sy_t, Sz_t = expect[1+n_nuclei:1+n_nuclei+3]

# ------------------------
# Build the collective single-flip nuclear state |1>_n (normalized)
# For n_nuclei=2 the single-flip states are |10> (flip n1) and |01> (flip n2)
# -----------------------------------------------------------------------
n00 = qutip.tensor(up, up)
n01 = qutip.tensor(up, down)
n10 = qutip.tensor(down, up)
n11 = qutip.tensor(down, down)

# create collective |1>_n = (sum_j a_j |flip_j>) / sqrt(sum |a_j|^2)
coeffs = []
states_single_flip = []
for k in range(n_nuclei):
    # note ordering: k=0 -> flip n1 -> state |10>, k=1 -> flip n2 -> |01>
    states_single_flip.append(n10 if k==0 else n01)
    coeffs.append(a_list[k])
norm = np.sqrt(np.sum(np.abs(coeffs)**2))
n1 = sum(c*s for c,s in zip(coeffs, states_single_flip)) / norm

# Many-body kets
phi_initial = qutip.tensor(down, n00)           # |↓>_e ⊗ |00>_n
phi_target  = qutip.tensor(up,   n1)            # |↑>_e ⊗ |1>_n  (collective)
phi_up_00   = qutip.tensor(up, n00)             # |↑>_e ⊗ |00>_n

# populations (overlaps) vs time
P_initial = np.zeros(len(states_t))
P_target  = np.zeros(len(states_t))
P_up00    = np.zeros(len(states_t))
for ti, psi in enumerate(states_t):
    P_initial[ti] = np.abs(phi_initial.overlap(psi))**2
    P_target[ti]  = np.abs(phi_target.overlap(psi))**2
    P_up00[ti]    = np.abs(phi_up_00.overlap(psi))**2

# full 8-state populations
probs = np.zeros((len(states_t), 2**(1+n_nuclei)))
for i, psi in enumerate(states_t):
    vec = psi.full().flatten()
    probs[i,:] = np.abs(vec)**2

# ------------------------
# PLOTTING
# ------------------------
# 1) Collective Rabi: initial <-> target <-> electron-up-empty (up,00)
plt.figure(figsize=(8,4))
plt.plot(tgrid*1e6, P_initial, label=r"$P(|\downarrow\rangle_e\otimes|00\rangle_n)$")
plt.plot(tgrid*1e6, P_target,  label=r"$P(|\uparrow\rangle_e\otimes|1\rangle_n)$")
plt.plot(tgrid*1e6, P_up00,    label=r"$P(|\uparrow\rangle_e\otimes|00\rangle_n)$")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Collective flip-flop populations (Ω/2π = {:.3f} MHz)".format(Omega_est/(2*np.pi)/1e6))
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 2) full 8 basis populations
plt.figure(figsize=(9,5))
for k in range(2**(1+n_nuclei)):
    plt.plot(tgrid*1e6, probs[:,k], label=f"|{k:03b}>")
plt.xlabel("time [µs]")
plt.ylabel("Population")
plt.title("Populations of all 8 basis states (electron,n1,n2)")
plt.legend(loc='upper right', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.show()

# 3) electron spin expectations
plt.figure(figsize=(8,4))
plt.plot(tgrid*1e6, Sx_t, label=r"$\langle S_x\rangle$")
plt.plot(tgrid*1e6, Sy_t, label=r"$\langle S_y\rangle$")
plt.plot(tgrid*1e6, Sz_t, label=r"$\langle S_z\rangle$")
plt.xlabel("time [µs]")
plt.ylabel("Spin expectation")
plt.title("Electron spin expectations")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 4) 2x4 amplitude heatmaps (initial and at time of max target)
def plot_amplitudes_matrix(psi, title="Amplitude matrix (probabilities)"):
    vec = psi.full().flatten()
    mat = vec.reshape((2, 2**n_nuclei))
    fig, ax = plt.subplots(figsize=(5,2.5))
    im = ax.imshow(np.abs(mat)**2, interpolation='nearest', aspect='auto')
    plt.colorbar(im, ax=ax, label='Population')
    ax.set_xticks(np.arange(2**n_nuclei))
    ax.set_xticklabels(["00","01","10","11"])
    ax.set_yticks([0,1])
    ax.set_yticklabels(["↑e","↓e"])
    ax.set_xlabel("nucleus state (n1 n2)")
    ax.set_ylabel("electron")
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

plot_amplitudes_matrix(states_t[0], title="Initial amplitudes")
imax = np.argmax(P_target)
plot_amplitudes_matrix(states_t[imax], title=f"Amplitudes at max target population (t={tgrid[imax]*1e6:.3f} µs)")

# 5) diagnostics
print("gmuB (rad/s/T):", gmuB)
print("gnumun (rad/s/T):", gnumun)
print("Used B0 (T):", B0)
if USE_B_RES:
    print("Computed B0_res (T):", B0_res)
print("Analytical Omega/(2π) [Hz]:", Omega_est/(2*np.pi))
print("Swap time t_swap (analytical) [µs]:", (np.pi / Omega_est)*1e6)
print("Max simulated target population:", P_target.max())
print("Time of max (µs):", tgrid[np.argmax(P_target)]*1e6)


a_list: [3141592.653589793, 3141592.653589793]
Analytical collective Rabi rate Omega/(2π) [Hz]: 707106.7811865475
Using fallback B0: 0.1 T
omega_eZ/(2π): 50.000 MHz, omega_NZ/(2π): 5.000 MHz


AttributeError: 'complex' object has no attribute 'full'